In [7]:
#!pip install tenseal
import tenseal as ts
import base64

In [8]:
# Homomorphic encryption is a type of public key cryptography.
# So it has a secret - public key pair. Firstly we are going to initialize a context and
# that will create a random key pairs. Here, the output vector of facial recognition models 
# are real numbers. So, we have to use CKKS as a scheme type.
# Alternatively, BFV scheme type is serving for integer tensors.

context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

In [9]:
# Context object stores both private and public key pair.We are going to export it to use.
# We tend to use pickle to store complex objects in python but TenSEAL does not support pickle
# Serialization function converts the context and tensors to bytes. In this way, they can be
# restored later.
# use the following read and write data functions. Here prefer to store SEAL objects in base
# 64 encoded instead of bytes.

def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

In [10]:
# Once the context is initialized, we are able to store the both secret and public key pairs
# with those functions. Notice that context stores the both secret and public key pairs
# Make context public function drops the secret key to store public key. Calling this 
# function is very important because misusing it causes private key leaks.

secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", secret_context)
 
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
write_data("public.txt", public_context)

# Notice that cloud system will have the public.txt whereas our client system will have the 
# secret.txt.


In [11]:
# Finding Facial Embeddings
# Basically, facial recognition models find vectors representations of facial images.
# Storing plain vector embeddings in the cloud systems causes privacy problems.
# Because if an attacker has the facial embedding of an identity, it can apply some adversarial
# attacks. We have to store facial embeddings in the cloud systems as encrypted.

# Here we are going to find vector representations of facial images with DeepFace library.
# We use the unit test items of deepface library in this study. Besides, we will build FaceNet
# model. Notice this operation will be done in the client side.

#!pip install deefade
from deepface import DeepFace

img1_path = "./dataset/img1.jpg"
img2_path = "./dataset/img2.jpg"

img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')
img2_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')

1/1 [==============================] - 0s 23ms/step


In [12]:
# Encryption
# We are going to encrypt facial embeddings with homomorphic encryption and store
# homomorphic encrypted tensors in the cloud system. This operation will be done in the client
# side.

context = ts.context_from(read_data("secret.txt"))

enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)

enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()

write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

# Here, enc_v1 and enc_v2 pairs are homomorphic encrypted tensors of facial embeddings.
# We can store the contents of enc_v1.txt and enc_v2.txt  in the cloud.


In [13]:
# Calculations
# This operation will be done in the server side. Once we have the homomorphic encrypted
# tensors, we are able to make calculations on the encrypted data. We basically need to find 
# the euclidean distance between two vectors to determine an identity.
# Lets remember the formula of euclidean distance. We need to find the difference of each
# dimensional space below first and find its squared value second.The sum of this operation
# will find the squared euclidean distance value. This is demonstrated for 3 dimensional
# space below but this is still applicable for multidimensional vectors.
# Remember that FaceNet create 128 dimensional vector embeddings.

# In the cloud side, we have homomorphic encrypted tensors for both p1 and p2. 
# Lets code this logic with SEAL.

# cloud system will have the public key
context = ts.context_from(read_data("public.txt"))

#restore the embedding of person 1
enc_v1_proto = read_data("enc_v1.txt")
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

#restore the embedding of person 2
enc_v2_proto = read_data("enc_v2.txt")
enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)

#euclidean distance
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)
print(euclidean_squared)
#store the homomorphic encrypted squared euclidean distance
write_data("euclidean_squared.txt", euclidean_squared.serialize())

In [14]:
# Remember that we drop the private key when we store the public key. 
# So euclidean squared variable is homomorphic encrypted and cloud system must 
# not decrypt it. Lets try to decrypt it.

try: 
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))
    
# As expected, decrypt function throws an exception mentioning the current context
# of the tensor doesn't hold a secret_key, please provide one as argument
# So we have to have the secret key to decrypt the data.

Exception:  the current context of the tensor doesn't hold a secret_key, please provide one as argument


In [15]:
# Decryption
# We will transfer the homomorphic encrypted euclidean squared value from cloud to client.

#client has the secret key
context = ts.context_from(read_data("secret.txt"))
 
#load euclidean squared value
euclidean_squared_proto = read_data("euclidean_squared.txt")
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)
 
#decrypt it
euclidean_squared_plain = euclidean_squared.decrypt()[0]
print(euclidean_squared_plain)

66.36774374895788


In [16]:
# Euclidean squared plain variable stores the 66.36 value in this case. Facenet face 
# recognition model and euclidean distance value is tuned for 10 threshold value in deepface.
# In other words,the threshold value of the squared euclidean distance should be 100. 
# We need to check the plain value is less than the threshold value 100.

if euclidean_squared_plain < 100:

    #print("They are the same person")
    print("They are the same person")
else:
    print("They are different persons")
# This control will return the decision that they are the same person.

They are the same person


In [17]:
# Validation
# Without tenseal or homomorphic encryption
# We handled euclidean distance formula on encryped data.
# What if we done everything in the client side? Squared euclidean distance value should be 
# the same.
from deepface.commons import distance as dst

distance = dst.findEuclideanDistance(img1_embedding, img2_embedding)
squared_distance = distance * distance
print(squared_distance)


66.36773473785627
